<a href="https://colab.research.google.com/github/bimarshak7/RAG-with-LLM/blob/main/Nepal_law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://lawcommission.gov.np/en/wp-content/uploads/2021/01/Constitution-of-Nepal.pdf -O sambhidhan.pdf -q
!wget https://moljpa.gov.np/en/wp-content/uploads/2018/12/Civil-procedure-code.pdf -O civil.pdf -q

In [ ]:
!pip install -q langchain langchain_community pypdf langchain_chroma sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00


In [ ]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = 'getpass.getpass()'

In [ ]:
from pprint import pprint

from typing import Optional, List
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
DOC_FILE = 'sambhidhan.pdf'
EMBEDDING_MODEL_NAME = "thenlper/gte-small"
MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
RAW_KNOWLEDGE_BASE = PyPDFLoader(DOC_FILE).load()

In [ ]:
RAW_KNOWLEDGE_BASE[2]

Document(page_content='www.lawcommission.gov.np  \n3 \n Part -1 \nPreliminary  \n1. Constitution as Fundamental Law: (1) This Constitution is the fundamental law of \nNepal. Any law inconsistent with this  Constitution shall, to the extent of such \ninconsistency, be void.  \n   (2) It shall be the duty of every person to  uphold this Constitution.   \n2. Sovereignty and State Power : The sovereignty and State  power of Nepal shall be  \nvested in the Nepali  people.  It shall be exercised in accordance with the provisions set \nforth in this Constitution.   \n3. Nation : All th e Nepali  people , with  multi -ethnic, multi -lingual, multi -religious, multi -\ncultural characteristics and in geographical diversities, and having common  aspirations \nand being united by a bond of allegiance to national independence, territorial integrity, \nnational interest and prosperity of Nepal, collectively constitute the nation.  \n4. State of Nepal : (1) Nepal is a n independent,  indivisible, so

In [ ]:
SEPARATORS = [
    r'Part -\d+',
    r'\\n\d+',
    r'\(\d+\)',
    r'\n',
    ' ',
    ''
]

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=512,     # Desired chunk size
#     chunk_overlap=100,   # Overlap between chunks
#     separators=SEPARATORS,
#     is_separator_regex=True
# )

In [ ]:
# splits = text_splitter.split_documents(RAW_KNOWLEDGE_BASE)
# len(RAW_KNOWLEDGE_BASE),len(splits)

In [ ]:
def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 5),
        add_start_index=True,
        strip_whitespace=True,
        separators=SEPARATORS,
        is_separator_regex=True

    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


In [ ]:
docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
len(docs_processed)

214

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(documents=docs_processed, embedding=embedding_model)

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

In [ ]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
# # Embed a user query in the same space
# user_query = "Explain child right of nepal"
# query_vector = embedding_model.embed_query(user_query)

In [ ]:
# print(f"\nStarting retrieval for {user_query=}...")
# retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
# print("\n==================================Top document==================================")
# print(retrieved_docs[0].page_content)
# print("==================================Metadata==================================")
# print(retrieved_docs[0].metadata)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=520,
)

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """You are a helpful assistant who can explain about the laws of Nepal.
Relevant parts of constitution is given in the context.
Respond only to the question asked, response should be concise and relevant to the question.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.
Question: {question}""",
    },
]

In [ ]:
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

In [ ]:
print(RAG_PROMPT_TEMPLATE)

<|system|>
You are a helpful assistant who can explain about the laws of Nepal. 
Relevant parts of constitution is given in the context.
Respond only to the question asked, response should be concise and relevant to the question.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.
Question: {question}</s>
<|assistant|>



In [ ]:
# retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
# context = "\nExtracted documents:\n"
# context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

# final_prompt = RAG_PROMPT_TEMPLATE.format(question="explain child right in nepal", context=context)

In [ ]:
# final_prompt

In [ ]:
# Redact an answer
# answer = READER_LLM(final_prompt)[0]["generated_text"]

In [ ]:
# print(answer)

In [ ]:
def answer_query():
  # Embed a user query in the same space
  print('AI: Ask me something, input bye to exit')
  while True:
    user_query = input('User:\t')
    if user_query=='bye':
      break
    query_vector = embedding_model.embed_query(user_query)
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)
    # Redact an answer
    answer = READER_LLM(final_prompt)[0]["generated_text"]
    print(f'AI\t: {answer}')

In [ ]:
answer_query()

AI: Ask me something, input bye to exit
User:	What is mentioned about supreme court in constitution of nepal
AI	: The following points related to the Supreme Court are mentioned in the extracts provided:

1. The Supreme Court shall exercise powers relating to justice in accordance with the Constitution, other laws, and recognized principles of justice (Article 126).
2. The Supreme Court shall be a court of record, and all courts and judicial institutions shall be under its jurisdiction, except as otherwise provided in the Constitution (Article 128).
3. The Supreme Court shall have the final authority to interpret the Constitution and laws (Article 128).
4. The Supreme Court may inspect, supervise, and give necessary directives to itself and other courts and judicial institutions under its jurisdiction regarding administration of justice or management (Article 128).
5. All shall abide by the orders or decisions made by the Supreme Court in the course of hearing a lawsuit (Article 126).


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


AI	: According to the extracts provided, children are individuals who fall under the following categories:

1. Those who have not reached the age of adulthood, as defined by law. This is implied in the definition of "child" in Document 1, where it states that every child shall have the right to name, birth registration, education, health, maintenance, proper care, sports, entertainment, overall personality development, elementary child development, child participation, and protection from hazardous work, child marriage, transportation, kidnapping, or taking hostage, abuse, exclusion, physical, mental, sexual, or other forms of exploitation or improper use, torture, and child-friendly justice.

2. Orphans, children with disabilities, conflict victims, displaced, and vulnerable children, who are entitled to special protection and facilities from the State, as stated in Document 1, clause (9).

3. Helpless children, as mentioned in Document 1, clause (9), who require special protection an